In [17]:
import pickle as pckl

In [18]:
import os

In [19]:
folder = "../models/trained_models"

In [20]:
pckl_list = os.listdir("../models/trained_models")

In [21]:
pckl_list

['pid=25736241310527711035.pkl',
 'pid=72632302122157640625.pkl',
 'pid=88653536218275668027.pkl',
 'pid=87205568023880642125.pkl']

In [23]:
with open(os.path.join(folder, pckl_list[-1]), 'rb') as f:
    test  = pckl.load(f)

In [24]:
test

{'train_both': True,
 'sparse_signal': True,
 'signal_strength': 1.0,
 'starting_folder': '../models/init_models/',
 'num_iters': 1,
 'seed': 0,
 'out_dir': '../models/trained_models/',
 'pid': '87205568023880642125',
 'losses_train': array([0.07745086]),
 'losses_test': array([0.58703369]),
 'accs_train': array([95.]),
 'accs_test': array([81.]),
 'model_weights': OrderedDict([('embed.weight',
               tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
                       [ 0.1116, -0.0059,  0.0143,  ..., -0.0309,  0.0140,  0.0126],
                       [ 0.1080,  0.1131,  0.0524,  ...,  0.1020, -0.1591,  0.0305],
                       ...,
                       [-1.0885, -0.0190,  0.0408,  ..., -0.1614,  0.2028, -0.2763],
                       [ 0.0206, -0.2081,  0.4568,  ...,  0.4254,  0.2714, -0.1388],
                       [-0.0134,  0.2994,  0.1973,  ...,  0.5958, -0.1408, -0.7416]],
                      device='cuda:0')),
              ('lstm.w

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
import sys
import os
sys.path.append('../../acd/acd')
sys.path.append('../../acd/visualization')
sys.path.append('../../acd/acd/util')
sys.path.append('../../acd/acd/scores')
sys.path.append('../../acd/acd/agglomeration')
import viz_1d as viz
import tiling_1d as tiling
import agg_1d as agg
import cd
import score_funcs
sys.path.append('../../acd')
sys.path.append('../../acd/dsets/sst')
from dsets.sst import dset
from dsets.sst.model import LSTMSentiment

# check out how two models differ
import torch.optim as O
import torch.nn as nn

from torchtext import data
from torchtext import datasets
from os.path import join

In [2]:

vector_cache =  '../data/.vector_cache/input_vectors.pt'
word_vectors ='glove.6B.300d'
batch_size = 50
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:

inputs = data.Field(lower= True)
answers = data.Field(sequential=False, unk_token=None)

train, dev, test = datasets.SST.splits(inputs, answers, fine_grained=False, train_subtrees=True,
                                       filter_pred=lambda ex: ex.label != 'neutral')

inputs.build_vocab(train, dev, test)

if os.path.isfile(vector_cache):
    inputs.vocab.vectors = torch.load(vector_cache)
else:
    inputs.vocab.load_vectors(word_vectors)
    makedirs(os.path.dirname(vector_cache))
    torch.save(inputs.vocab.vectors,vector_cache)
answers.build_vocab(train)

train_iter, dev_iter, test_iter = data.BucketIterator.splits(
    (train, dev, test), batch_size=batch_size, device=torch.device(0))

In [4]:
model_path = "../models/init_models"
model_list = os.listdir(model_path)
model1 = torch.load(join(model_path, model_list[0]), map_location=torch.device(0)).eval()

model2 = torch.load(join(model_path, model_list[1]), map_location=torch.device(0)).eval()

In [5]:

is_correct = np.empty((2,len(dev)))

In [7]:
with torch.no_grad():
    # check out how two models differ
    import torch.optim as O
    import torch.nn as nn
    criterion = nn.CrossEntropyLoss()
    n_dev_correct, dev_loss = 0, 0
    cur_idx = 0
    for dev_batch_idx, dev_batch in enumerate(dev_iter):



        answer1 = model1(dev_batch)
        answer2  = model2(dev_batch)

        is_correct[0, cur_idx:cur_idx + dev_batch.label.shape[0]] = (torch.max(answer1, 1)[1].view(dev_batch.label.size()).data == dev_batch.label.data).cpu().numpy()
        is_correct[1, cur_idx:cur_idx + dev_batch.label.shape[0]] = (torch.max(answer2, 1)[1].view(dev_batch.label.size()).data == dev_batch.label.data).cpu().numpy()

        cur_idx+=dev_batch.label.shape[0]
        n_dev_correct += (((torch.max(answer1, 1)[1].view(dev_batch.label.size()).data == dev_batch.label.data)+
        (torch.max(answer2, 1)[1].view(dev_batch.label.size()).data == dev_batch.label.data))>=1).sum()
        dev_loss = criterion(answer1, dev_batch.label)
    dev_acc = 100. * n_dev_correct / len(dev)

In [8]:
is_correct -= is_correct.mean(axis=1)[:,np.newaxis]

In [9]:
is_correct /=(is_correct.std(axis=1)[:,np.newaxis])

In [10]:
np.cov(is_correct[0], is_correct[1])

array([[1.00114811, 0.68973568],
       [0.68973568, 1.00114811]])